In [2]:
from img2dataset import download
import shutil
import multiprocessing
import json
import os
from datasets import Dataset, load_from_disk
from pathlib import Path
from PIL import Image

In [3]:
train_captions_path = '/root/autodl-tmp/data/annotations/captions_train2017.json'
val_captions_path = '/root/autodl-tmp/data/annotations/captions_val2017.json'
train_path = '/root/autodl-tmp/data/train2017'
val_path = '/root/autodl-tmp/data/val2017'

save_to_path = "/root/autodl-tmp/data/grayscale_image_512_100k"
cache_path = "/root/autodl-tmp/data/cache_512_100k"

img_path = train_path
caption_path = train_captions_path

with open(caption_path) as f:
    annotations = json.load(f)["annotations"]
print(annotations[0:10])
print(len(annotations))

[{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}, {'image_id': 322141, 'id': 49, 'caption': 'A room with blue walls and a white sink and door.'}, {'image_id': 16977, 'id': 89, 'caption': 'A car that seems to be parked illegally behind a legally parked car'}, {'image_id': 106140, 'id': 98, 'caption': 'A large passenger airplane flying through the air.'}, {'image_id': 106140, 'id': 101, 'caption': 'There is a GOL plane taking off in a partly cloudy sky.'}, {'image_id': 322141, 'id': 109, 'caption': 'Blue and white color scheme in a small bathroom.'}, {'image_id': 322141, 'id': 121, 'caption': 'This is a blue and white bathroom with a wall sink and a lifesaver on the wall.'}, {'image_id': 322141, 'id': 163, 'caption': 'A blue boat themed bathroom with a life preserver on the wall'}, {'image_id': 203564, 'id': 181, 'caption': 'The bike has a clock as a tire.'}, {'image_id': 16977, 'id': 185, 'caption': 'two cars parked on the sidewalk on the 

In [4]:
image_folder = Path(img_path)
size = 512
def entry_for_id(image_folder, image_id, caption):
    file_name = str(image_id).zfill(12) + '.jpg'
    img = Image.open(image_folder / file_name)
    img = img.resize((size, size))
    gray_img = img.convert('L')
    
    return {
        "image": img,
        "grayscale_image": gray_img,
        "caption": caption
    }

            
max_images = 100000


def generate_entries():
    index = 0
    for annotation in annotations:
        image_id = annotation['image_id']
        caption = annotation['caption']
        yield entry_for_id(image_folder, image_id, caption)
        index += 1
        if index >= max_images:
            break

# def g():
#     i = 0
#     for i in range(2000):
#         yield {
#         "image": i,
#         "grayscale_image": i,
#         "caption": i
#     }
ds = Dataset.from_generator(generate_entries, cache_dir=cache_path)
# ds = Dataset.from_generator(generate_entries, output_types=tf.int32)
# ds = Dataset.from_generator(g)
ds.save_to_disk(save_to_path)

Generating train split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/120 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
# visualize images during training
from PIL import Image
import random

with open(val_captions_path) as f:
    annotations = json.load(f)["annotations"]
    
randomSamp = random.sample(range(500), 5)

for i in randomSamp:
    img_id = annotations[i]['image_id']
    caption = annotations[i]['caption']
    image = Image.open('/root/autodl-tmp/data/val2017/'+str(img_id).zfill(12) + '.jpg')
    new_im = image.resize((size, size))
    new_im = new_im.convert('L')
    new_im.save('/root/diffusers/examples/controlnet/samples/'+ str(img_id) + '.jpg')
    print(caption)

In [5]:
from PIL import Image
import random
with open(val_captions_path) as f:
    annotations = json.load(f)["annotations"]
    
randomSamp = random.sample(range(500), 10)

for i in randomSamp:
    img_id = annotations[i]['image_id']
    caption = annotations[i]['caption']
    image = Image.open('/root/autodl-tmp/data/val2017/'+str(img_id).zfill(12) + '.jpg')
    new_im = image.resize((512, 512))
    new_im.save('/root/diffusers/examples/controlnet/samples/512/'+ 'color' + str(img_id) + '.jpg')
    new_im = new_im.convert('L')
    new_im.save('/root/diffusers/examples/controlnet/samples/512/'+ str(img_id) + '.jpg')
    print(str(img_id) + caption)

226408Child laying down with arms extended in the air.
99054An airplane parked at the runway is being serviced by workers.
249025A beautiful yellow rose is seen in a small vase.
561256A woman in a yellow bathroom is holding a camera.
492937The bicycle is locked to the metal pole as the young man waits to board the mass  transit train.
423971a bathroom with a fancy toilet in it
44652An single passanger in an airplane flying through the sky
63740A laptop computer and a desktop computer on a white desk
140420A motorcycle is parked on a gravel road in a forest by a stream.
179765A motorcycle with its brake extended standing outside
